# Sciware: Python Packaging

This notebook is a cut down version of Dan Foreman-Mackey's [Blog Post](https://dfm.io/posts/autocorr/).
If you're curious what the calculations here are doing or what they are for, see that!

We use it here as an example of a common workflow: Jupyter Notebooks combining some 
calculation (which maybe you want to re-use!) with a concrete data set and/or visualization.

We will iteratively move parts of this notebook into a package while keeping the rest of the
notebook working the whole time.

Some useful links:
- https://setuptools.pypa.io/en/latest/userguide/quickstart.html
- https://setuptools.pypa.io/en/latest/userguide/pyproject_config.html
- https://packaging.python.org/en/latest/guides/writing-pyproject-toml/
- https://packaging.python.org/en/latest/tutorials/packaging-projects/

In [ ]:
# Google Colab: Uncomment and run the next line!
#!pip install matplotlib scipy emcee autograd numpy celerite tqdm build twine

## Generating data (fake MCMC chains, in this case)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(123456)

# Build the celerite model:
import celerite
from celerite import terms

kernel = terms.RealTerm(log_a=0.0, log_c=-6.0)
kernel += terms.RealTerm(log_a=0.0, log_c=-2.0)

# The true autocorrelation time can be calculated analytically:
true_tau = sum(2 * np.exp(t.log_a - t.log_c) for t in kernel.terms)
true_tau /= sum(np.exp(t.log_a) for t in kernel.terms)
true_tau

# Simulate a set of chains:
gp = celerite.GP(kernel)
t = np.arange(2000000)
gp.compute(t)
y = gp.sample(size=32)

# Let's plot a little segment with a few samples:
plt.plot(y[:3, :300].T)
plt.xlim(0, 300)
plt.xlabel("step number")
plt.ylabel("$f$")
plt.title("$\\tau_\mathrm{{true}} = {0:.0f}$".format(true_tau), fontsize=14);

In [ ]:
!pip install -e ./autocorr/
# after the first time you run the above command, you may need to restart the ipython kernel
# future reloads can be handled by %autoreload, see https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 3

### Empirical autocorrelation function

In [ ]:
from autocorr.estimators import autocorrelation_function_1d
# could also import autocorr and use autocorr.estimators.autocorrelation_function_1d ...


# Make plots of ACF estimate for a few different chain lengths
window = int(2 * true_tau)
tau = np.arange(window + 1)
f0 = kernel.get_value(tau) / kernel.get_value(0.0)

# Loop over chain lengths:
fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharex=True, sharey=True)
for n, ax in zip([10, 100, 1000], axes):
    nn = int(true_tau * n)
    ax.plot(tau / true_tau, f0, "k", label="true")
    ax.plot(tau / true_tau, autocorrelation_function_1d(y[0, :nn])[: window + 1], label="estimate")
    ax.set_title(r"$N = {0}\,\tau_\mathrm{{true}}$".format(n), fontsize=14)
    ax.set_xlabel(r"$\tau / \tau_\mathrm{true}$")

axes[0].set_ylabel(r"$\rho_f(\tau)$")
axes[-1].set_xlim(0, window / true_tau)
axes[-1].set_ylim(-0.05, 1.05)
axes[-1].legend(fontsize=14);

### Estimating autocorrelation time


In [ ]:
from autocorr.estimators import integrated_autocorrelation_time_averaged, integrated_autocorrelation_time_ensemble


# Compute the estimators for a few different chain lengths
N = np.exp(np.linspace(np.log(100), np.log(y.shape[1]), 10)).astype(int)
gw2010 = np.empty(len(N))
new = np.empty(len(N))
for i, n in enumerate(N):
    gw2010[i] = integrated_autocorrelation_time_averaged(y[:, :n])
    new[i] = integrated_autocorrelation_time_ensemble(y[:, :n])

# Plot the comparisons
plt.loglog(N, gw2010, "o-", label="G&W 2010")
plt.loglog(N, new, "o-", label="DFM 2017")
ylim = plt.gca().get_ylim()
plt.plot(N, N / 50.0, "--k", label=r"$\tau = N/50$")
plt.axhline(true_tau, color="k", label="truth", zorder=-100)
plt.ylim(ylim)
plt.xlabel("number of samples, $N$")
plt.ylabel(r"$\tau$ estimates")
plt.legend(fontsize=14);

## A more realistic example

Now, let's run an actual Markov chain and test these methods using those samples.
So that the sampling isn't completely trivial, we'll sample a multimodal density in three dimensions.

In [ ]:
import emcee

def log_prob(p):
    return np.logaddexp(-0.5 * np.sum(p**2), -0.5 * np.sum((p - 4.0) ** 2))


sampler = emcee.EnsembleSampler(32, 3, log_prob)
sampler.run_mcmc(
    np.concatenate((np.random.randn(16, 3), 4.0 + np.random.randn(16, 3)), axis=0),
    50000,
    progress=True,
);

In [ ]:
chain = sampler.get_chain()[:, :, 0].T

plt.hist(chain.flatten(), 100)
plt.gca().set_yticks([])
plt.xlabel(r"$\theta$")
plt.ylabel(r"$p(\theta)$");

Convergence of the ACT estimate as chains get longer

In [ ]:
# Compute the estimators for a few different chain lengths
N = np.exp(np.linspace(np.log(100), np.log(chain.shape[1]), 10)).astype(int)
gw2010 = np.empty(len(N))
new = np.empty(len(N))
for i, n in enumerate(N):
    gw2010[i] = integrated_autocorrelation_time_averaged(chain[:, :n])
    new[i] = integrated_autocorrelation_time_ensemble(chain[:, :n])

# Plot the comparisons
plt.loglog(N, gw2010, "o-", label="G\&W 2010")
plt.loglog(N, new, "o-", label="DFM 2017")
ylim = plt.gca().get_ylim()
plt.plot(N, N / 50.0, "--k", label=r"$\tau = N/50$")
plt.ylim(ylim)
plt.xlabel("number of samples, $N$")
plt.ylabel(r"$\tau$ estimates")
plt.legend(fontsize=14);

### Estimating ACT using a autoregression model

In [ ]:
from autocorr.estimators import integrated_autocorrelation_time_learned

# Calculate the estimate for a set of different chain lengths
ml = np.empty(len(N))
ml[:] = np.nan
for j, n in enumerate(N[1:8]):
    i = j + 1
    thin = max(1, int(0.05 * new[i]))
    ml[i] = integrated_autocorrelation_time_learned(chain[:, :n], thin=thin)

In [ ]:
# Plot the comparisons
plt.loglog(N, gw2010, "o-", label="G\&W 2010")
plt.loglog(N, new, "o-", label="DFM 2017")
plt.loglog(N, ml, "o-", label="DFM 2017: ML")
ylim = plt.gca().get_ylim()
plt.plot(N, N / 50.0, "--k", label=r"$\tau = N/50$")
plt.ylim(ylim)
plt.xlabel("number of samples, $N$")
plt.ylabel(r"$\tau$ estimates")
plt.legend(fontsize=14);

In this case, this estimate seems to be robust even for very short chains with $N \sim \tau$.